In [373]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import json
from sklearn.preprocessing import StandardScaler
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [467]:
dfp = pd.read_csv('data-set/personal_valid.csv')
dfo = pd.read_csv('data-set/other_valid.csv')
dfp = dfp.rename(columns={'Unnamed: 0': 'ID'})
dfo = dfo.rename(columns={'Unnamed: 0': 'ID'})

Táto časť je skopírovaná z predchadzajúceho notebooku 'Prieskumná_analýza.ipynb'. Spojí dataset other a personal do jedného.

In [468]:
# merges duplicates
def merge(name):
    df = dfo_duplicates.loc[dfo_duplicates['name'] == name]
    return df.groupby(['name'], as_index=False).first()

dfo_duplicates = dfo[dfo.duplicated(['name'], keep=False)]
dfo_unique = dfo.drop_duplicates(subset=["name"], keep=False)
merged = []

for name in dfo_duplicates['name'].unique():
    merged.append(merge(name))
    
dfo_unique = dfo_unique.append(merged)

df = pd.merge(dfp, dfo_unique, on='name')
df = df.drop(columns=['address_y', 'ID_y'])
df = df.rename(columns={"ID_x": "ID", "address_x": "address"})

Pridame nove stlpce na rozbalenie hodnot v medical_info.

In [469]:
for i, row in df.iterrows():
    if not pd.isnull(df.at[i, 'medical_info']):
        df.at[i, 'medical_info'] = json.loads(df["medical_info"][i].replace("\'", "\""))

In [470]:
df_med_info = df["medical_info"].apply(pd.Series)
df_med_info = df_med_info.drop(0, 1)

df_final = pd.concat([df, df_med_info], axis = 1).drop("medical_info", axis = 1)

Nahradenie pohlavi ' Female', ' Male' za 'Female', 'Male'.

In [471]:
df['sex'] = df['sex'].replace(' Male', 1)
df['sex'] = df['sex'].replace(' Female', 0)

Nahradenie pregnant 't' 'T' 'F' 'f' za 'True a 'False'

In [473]:
df['pregnant'] = df['pregnant'].replace(regex='(?i)f.*', value=0)
df['pregnant'] = df['pregnant'].replace(regex='(?i)t.*', value=1)

Upravenie adresy

In [480]:
df['address'].replace(regex='\r\n.*', value=0)

'7910 Rosales Plain Apt. 454\r\nPort Carl, GA 69839'